## GEO 877 - Spatial Algorithms 
### Group Kirsteina 
#### Tamara, Joya, Andrejs, Djordje

### Data


#### Description
For our analysis, we will be using the following data:
- Park data - Grünfläschen - Stadt Zürich - **BOUNDARY DATA**
    - 4 data types: Parks, Sports Areas, Cemeteries, Other
    - We will use Parks, Sports Areas, and Cemeteries as designated "parks"
    - We are considering adding Forests to the "parks" set
        - Could someone please download it, I can't: https://www.stadt-zuerich.ch/geodaten/download/111
    - **Can we find a shapefile somewhere that has just green areas as polygons?** we can use it to calculate % areas


- Fountain data - **EVALUATION DATA**
    - Brunnen - fountains for heat relief
    - Stillgewässer - fountains for drinking 
    - **Can we find a shapefile somewhere that has just water areas as polygons?** we can use it to calculate % areas


- ZüriWC data - **EVALUATION DATA**
    - Location of publicly accessible WCs


- Spielpark data - **EVALUATION DATA**
    - Location of kids' playgrounds


- LIDAR data - Canopy Height - **EVALUATION DATA**
    - Download link: https://www.stadt-zuerich.ch/geodaten/download/Baumhoehen_2022__CHM_aus_Lidar_ 


- Socialshilfe data - **EVALUATION DATA**
    - Data on Social Assistance Quotas - need additional clarity whether this is percent of residents that receive Social Assistance or else


#### Data Structure

| PolygonID | Polygon Info | Green Area | Water Area | WC | Fountain | Playground | Socialshilfe | Canopy Height |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| X1 | coordinates | % of park | % of park | yes/no boolean | yes/no boolean | yes/no boolean | quota* | height* |

*needs more thought and discussion


### Analysis Workflow - Preliminary


1. Create park polygons:
    1. Download Wald data
    2. Remove "other" category
    3. Add Wald data
    4. Finalize all polygons - ensure all lines are cohesive
2. LIDAR Data:
    1. Prepare LIDAR data
    2. Extract only relevant data for Parks (Union of Park extent and LIDAR) to remove excess
3. Calculations:
    1. Park area as green space - %
    2. Park area as water space - %
4. Data Merge:
    1. Green area - join data to each park
    2. Water area - join data to each park
    3. Socialshilfe - join data to each park
5. Point in Polygon:
    1. WC - each park receives a numeric value
    2. Fountain - each park receives a numeric value
    3. Playground - each park receives a numeric value
6. Point in Polygon - translated:
    1. For each (WC, Fountain, Playground) assign a yes/no as result of a logical test loop (0 = no, 0> yes)
7. LIDAR Data
    1. **Determine a way to use this data** that is in line with the paper
8. Clustering


### Production

In [14]:
#Packages

from geospatial import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.geometry import shape
from shapely.ops import unary_union


#### 1. Polygon creation

In [3]:
#Data Import

parks = gpd.read_file('data/grunflaschen_csv/data/gsz.gruenflaechen.csv')
#parks['produkt'].unique()

#forests = pd.read_csv
#...

In [4]:
#Data Cleaning
#filter out 640 Weitere Freiräume
filtered_park = []
for x, y in parks.iterrows():
    if y['produkt'] != "640 Weitere Freiräume":
        filtered_park.append(y)

dfParks = pd.DataFrame(filtered_park)

#dfParks['produkt'].unique()
#parks.info()

# Function to parse WKT and extract coordinates
def parse_wkt_to_points(wkt_string):
    geometry = wkt.loads(wkt_string)  # Parse WKT string into a Shapely geometry
    if isinstance(geometry, Polygon):
        # If it's a single Polygon, extract its exterior coordinates
        return [(coord[0], coord[1]) for coord in geometry.exterior.coords]
    elif isinstance(geometry, MultiPolygon):
        # If it's a MultiPolygon, process the largest polygon by area
        largest_polygon = max(geometry.geoms, key=lambda p: p.area)
        return [(coord[0], coord[1]) for coord in largest_polygon.exterior.coords]
    else:
        # Handle unexpected geometry types (e.g., Point, LineString)
        raise ValueError(f"Unsupported geometry type: {type(geometry)}")

# Apply the function to the geometry column
dfParks['parsed_geometry'] = dfParks['geometry'].apply(parse_wkt_to_points)

# Check the parsed geometry
print(dfParks['parsed_geometry'].head())
print(dfParks.head())


0    [(2680607.0, 1250699.1), (2680607.0, 1250699.2...
1    [(2680616.2, 1250717.9), (2680616.8, 1250724.2...
2    [(2679110.2, 1247857.2), (2679116.2, 1247883.9...
3    [(2679116.5, 1247800.5), (2679117.2, 1247806.9...
4    [(2679117.5, 1247856.1), (2679118.2, 1247860.0...
Name: parsed_geometry, dtype: object
  objektidentifikator         pflegeareal          produkt erfassungseinheit  \
0                5956           Müseliweg  610 Parkanlagen   611 Parkanlagen   
1                5958           Müseliweg  610 Parkanlagen   611 Parkanlagen   
2                5960  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   
3                5962  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   
4                5964  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   

  objectid            pflegeeinheit pflegestufe  \
0        1         6018 Blumenwiese           3   
1        2  6018 Strauchbepflanzung           3   
2        3         2736 Blumenwiese           2   
3  

In [5]:
#ALGORITHMS 

from numpy import sqrt, radians, arcsin, sin, cos

####### Point #######

class Point():
    # initialise
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
    
    # representation
    def __repr__(self):
        return f'Point(x={self.x:.2f}, y={self.y:.2f})'

        # Test for equality between Points
    def __eq__(self, other): 
        if not isinstance(other, Point):
            # don't attempt to compare against unrelated types
            return NotImplemented
        return self.x == other.x and self.y == other.y

    # We need this method so that the class will behave sensibly in sets and dictionaries
    def __hash__(self):
        return hash((self.x, self.y))
    
    # calculate Euclidean distance between two points
    def distEuclidean(self, other):
        return sqrt((self.x-other.x)**2 + (self.y-other.y)**2)
    
    # calculate Manhattan distance between two points
    def distManhattan(self, other):
        return abs(self.x-other.x) + abs(self.y-other.y)

    # Haversine distance between two points on a sphere - requires lat/lng converted to radians
    def distHaversine(self, other):
        r = 6371000  # Earth's radius in metres (will return result in metres)
        phi1 = radians(self.y) # latitudes
        phi2 = radians(other.y)
        lam1 = radians(self.x) # longitudes
        lam2 = radians(other.x)

        d = 2 * r * arcsin(sqrt(sin((phi2 - phi1)/2)**2 + 
                                      cos(phi1) * cos(phi2) * sin((lam2 - lam1)/2)**2))
        return d   

    
    # Calculate determinant with respect to three points. Note the order matters here - we use it to work out left/ right in the next method
    def __det(self, p1, p2):
        det = (self.x-p1.x)*(p2.y-p1.y)-(p2.x-p1.x)*(self.y-p1.y)       
        return det

    def leftRight(self, p1, p2):
        # based on GIS Algorithms, Ch2, p11-12, by Ningchuan Xiao, publ. 2016
        # -ve: this point is on the left side of a line connecting p1 and p2
        #   0: this point is collinear
        # +ve: this point is on the right side of the line
        side = int(self.__det(p1, p2))
        if side != 0:
            side = side/abs(side)  # will return 0 if collinear, -1 for left, 1 for right
        return side
    
####### PointGroup #######
class PointGroup(): 
    # initialise
    def __init__(self, data=None, xcol=None, ycol=None):
        self.points = []
        self.size = len(data)
        for d in data:
            self.points.append(Point(d[xcol], d[ycol]))
    
    # representation
    def __repr__(self):
        return f'PointGroup containing {self.size} points' 
 
    # create index of points in group for referencing
    def __getitem__(self, key):
        return self.points[key]
    

####### Polygon #######
class Polygon(PointGroup):  
    # initialise
    def __init__(self, data=None, xcol=None, ycol=None):
        self.points = []
        self.size = len(data)
        for d in data:
            self.points.append(Point(d[xcol], d[ycol]))
        self.bbox = Bbox(self)
        
    # representation
    def __repr__(self):
        return f'Polygon PointGroup containing {self.size} points' 
  
    # test if polygon is closed: first and last point should be identical
    def isClosed(self):
        start = self.points[0]
        end = self.points[-1]
        return start == end

    def removeDuplicates(self):
        oldn = len(self.points)
        self.points = list(dict.fromkeys(self.points)) # Get rid of the duplicates
        self.points.append(self.points[0]) # Our polygon must have one duplicate - we put it back now
        n = len(self.points)
        print(f'The old polygon had {oldn} points, now we only have {n}.')
        
        # find area and centre of the polygon
    # - based on GIS Algorithms, Ch.2 p9-10, by Ningchuan Xiao, publ. 2016 
    
    def __signedArea(self):  # used for both area and centre calculations - this is a private method (only used within the class)
        a = 0
        xmean = 0
        ymean = 0
        for i in range(0, self.size-1):
            ai = self[i].x * self[i+1].y - self[i+1].x * self[i].y
            a += ai
            xmean += (self[i+1].x + self[i].x) * ai
            ymean += (self[i+1].y + self[i].y) * ai

        a = a/2.0   # signed area of polygon (can be a negative)
    
        return a, xmean, ymean
    
    def area(self):
        a, xmean, ymean = self.__signedArea()
        area = abs(a)   # absolute area of polygon
        
        return area

    def centre(self):
        a, xmean, ymean = self.__signedArea() # note we use the signed area here
        centre = Point(xmean/(6*a), ymean/(6*a)) # centre of polygon 
        return centre

    def containsPoint(self, p):
        if (self.bbox.containsPoint(p) == False):
            return False
        
        # Solution, as discussed in lecture, added here
        ray = Segment(p, Point(self.bbox.ur.x+1, p.y))
        count = 0
        
        for i in range(0, self.size-1):
            start = self[i]
            end = self[i+1]
            if (start.y != end.y): 
                if ((p.x < start.x) and (p.y == start.y)): 
                    count = count + 1
                else:
                    s = Segment(start, end)
                    if s.intersects(ray):
                        count = count + 1
              
        if (count%2 == 0):
            return False           

        return True

In [9]:
#Calculate area of each Park Polygon

def calculate_polygon_area(polygon_points):
    """
    Calculate the area of a polygon using the Shoelace formula.
    :param polygon_points: List of tuples representing the vertices of the polygon [(x1, y1), (x2, y2), ...]
    :return: Area of the polygon
    """
    n = len(polygon_points)
    if n < 3:
        return 0  # A polygon must have at least 3 points

    area = 0
    for i in range(n):
        x1, y1 = polygon_points[i]
        x2, y2 = polygon_points[(i + 1) % n]  # Wrap around to the first point
        area += x1 * y2 - x2 * y1

    return abs(area) / 2

# Apply the algorithm to the DataFrame
def add_polygon_area_column(df):
    """
    Add a new column 'poly_area' to the DataFrame with the calculated area for each polygon.
    :param df: DataFrame containing a column 'parsed_geometry' with lists of tuples representing polygons
    :return: DataFrame with the new 'poly_area' column
    """
    df['poly_area'] = df['parsed_geometry'].apply(calculate_polygon_area)
    return df

# Example usage
# Assuming dfParks is already loaded and contains a 'parsed_geometry' column
# dfParks = pd.read_csv('path_to_your_file.csv')  # Example of loading the DataFrame
dfParks = add_polygon_area_column(dfParks)

# Print the DataFrame to verify the new column
#print(dfParks[['parsed_geometry', 'poly_area']].head())

print(dfParks.head())

  objektidentifikator         pflegeareal          produkt erfassungseinheit  \
0                5956           Müseliweg  610 Parkanlagen   611 Parkanlagen   
1                5958           Müseliweg  610 Parkanlagen   611 Parkanlagen   
2                5960  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   
3                5962  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   
4                5964  Grünzug Bachwiesen  610 Parkanlagen   611 Parkanlagen   

  objectid            pflegeeinheit pflegestufe  \
0        1         6018 Blumenwiese           3   
1        2  6018 Strauchbepflanzung           3   
2        3         2736 Blumenwiese           2   
3        4      2736 Gebrauchsrasen           3   
4        5         2736 Blumenwiese           2   

                                            geometry  \
0  POLYGON ((2680607 1250699.1,2680607 1250699.2,...   
1  POLYGON ((2680616.2 1250717.9,2680616.8 125072...   
2  POLYGON ((2679110.2 1247857.2,2679116.2 1

In [15]:
#Dissolving polygons into one by Pflegeareal (Name)

# Convert the 'geometry' column to Shapely geometries
dfParks['geometry'] = dfParks['geometry'].apply(shape)

# Group by 'pflegeareal' and combine geometries
grouped = dfParks.groupby('pflegeareal').agg({
    'geometry': lambda x: unary_union(x),  # Combine geometries into a single MultiPolygon
    'objektidentifikator': 'first',        # Keep the first value
    'produkt': 'first',                    # Keep the first value
    'object id': 'first',                  # Keep the first value
    'pflegeeinheit': lambda x: ', '.join(x),  # Combine into a single string
    'poly_area': 'sum'                     # Sum the areas
}).reset_index()

# Add an area column
grouped['area'] = grouped['geometry'].apply(lambda geom: geom.area)

# Sort by area in descending order and keep the largest geometry for each 'pflegeareal'
largest_geometry = grouped.sort_values(by='area', ascending=False).drop_duplicates(subset='pflegeareal', keep='first')

# Save the result to a CSV file
largest_geometry[['pflegeareal', 'area']].to_csv('data/largest_geometry.csv', index=False)

# Print the result
print(largest_geometry)

AttributeError: 'str' object has no attribute 'get'

In [ ]:
#Joya's convert to geodataframe 
gdfParks = gpd.GeoDataFrame(dfParks, geometry="geometry", crs="EPSG:2056") 
#group and dissolve by 'pflegeareal'
gdf_grouped = gdfParks.dissolve(by="pflegeareal")
# Reset the index to make 'pflegeareal' a column again
gdf_grouped = gdf_grouped.reset_index()
# Add an area column
gdf_grouped['area'] = gdf_grouped.geometry.area
# Sort by area in descending order and keep the largest geometry for each 'pflegeareal', so that biggest area is kept
largest_geometry = gdf_grouped.sort_values(by='area', ascending=False).drop_duplicates(subset='pflegeareal', keep='first')
# Save the result to a new csv
largest_geometry.to_file('data/largest_geometry.shp')
largest_geometry[['pflegeareal', 'area']].to_csv('data/largest_geometry.csv', index=False)

# Print the result
print(largest_geometry)
#hey


TypeError: Input must be valid geometry objects: POLYGON ((2680607 1250699.1,2680607 1250699.2,2680607 1250699.5,2680607 1250699.6,2680607.2 1250699.9,2680607.2 1250700,2680607.2 1250700.2,2680607.2 1250700.1,2680607.5 1250700.1,2680607.5 1250700,2680607.8 1250699.9,2680607.8 1250699.9,2680608 1250699.8,2680608 1250699.6,2680608.2 1250699.5,2680608.2 1250699.2,2680608.5 1250699.1,2680608.5 1250699,2680608.8 1250698.9,2680610.8 1250694.8,2680611.2 1250693.6,2680611.8 1250692.6,2680613 1250689,2680612.8 1250688.4,2680607.5 1250692.8,2680607.2 1250695.2,2680607 1250699.1))

#### 2. LiDAR data preparation

#### 3. % Space Calculations (I think that we can make this an algorithm, too)

#### 4. Data Merging/Joining

#### 5. Point in Polygon (Algorithm)

### 6. Socialshilfe data prep

#### X. Clustering (We could attempt an algorithm)